In [1]:
%load_ext autoreload

In [7]:
%matplotlib inline
import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

import pandas as pd
import numpy as np

import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split, KFold
from sklearn.grid_search import GridSearchCV
from sklearn.decomposition import NMF

import transform_utils as tu
from kaggle_helper import KaggleHelper

In [8]:
kh = KaggleHelper("submissions - matrix factorization.db", "predictions")

table submissions already exists
table validation_scores already exists


In [9]:
%aimport transform_utils
%aimport kaggle_helper

In [10]:
%aimport

Modules to reload:
kaggle_helper transform_utils

Modules to skip:



In [11]:
raw = pd.read_csv("data/train.csv")
print("Unique UPC:", len(raw.Upc.unique()))
print("Unique DepartmentDescription:", len(raw.DepartmentDescription.unique()))
print("Unique FinelineNumber:", len(raw.FinelineNumber.unique()))
raw.head()

Unique UPC: 97715
Unique DepartmentDescription: 69
Unique FinelineNumber: 5196


,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,999,5,Friday,68113152929,-1,FINANCIAL SERVICES,1000
1,30,7,Friday,60538815980,1,SHOES,8931
2,30,7,Friday,7410811099,1,PERSONAL CARE,4504
3,26,8,Friday,2238403510,2,PAINT AND ACCESSORIES,3565
4,26,8,Friday,2006613744,2,PAINT AND ACCESSORIES,1017


In [12]:
raw = pd.read_csv("data/train.csv")
raw['Returns'] = raw.ScanCount.map(lambda x: abs(x) if x < 0 else 0)
raw.ScanCount = raw.ScanCount.map(lambda x: 0 if x < 0 else x)

In [13]:
raw.head()

,TripType,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber,Returns
0,999,5,Friday,68113152929,0,FINANCIAL SERVICES,1000,1
1,30,7,Friday,60538815980,1,SHOES,8931,0
2,30,7,Friday,7410811099,1,PERSONAL CARE,4504,0
3,26,8,Friday,2238403510,2,PAINT AND ACCESSORIES,3565,0
4,26,8,Friday,2006613744,2,PAINT AND ACCESSORIES,1017,0


In [24]:
start = datetime.datetime.now()
%autoreload
group_cols = ['Weekday']
keep_cols = ['VisitNumber', 'ScanCount', 'Returns']
df = tu.preprocess_walmart(raw, group_cols, keep_cols, True)

print("Total Time:", (datetime.datetime.now() - start).total_seconds())

['VisitNumber', 'ScanCount', 'Returns', 'Weekday_Friday', 'Weekday_Monday', 'Weekday_Saturday', 'Weekday_Sunday', 'Weekday_Thursday', 'Weekday_Tuesday', 'Weekday_Wednesday']
Total Time: 13.477076


In [25]:
df.columns

Index(['Weekday_Sunday', 'Weekday_Saturday', 'Weekday_Wednesday',
       'Weekday_Thursday', 'TripType', 'VisitNumber', 'Weekday_Tuesday',
       'ScanCount', 'Returns', 'Weekday_Monday', 'Weekday_Friday',
       'Weekday_Sunday', 'Weekday_Saturday', 'Weekday_Wednesday',
       'Weekday_Thursday', 'VisitNumber', 'Weekday_Tuesday', 'ScanCount',
       'Returns', 'Weekday_Monday', 'Weekday_Friday'],
      dtype='object')

In [20]:
X = df.drop("TripType", axis=1).drop("VisitNumber", axis=1)
sorted_columns = sorted(X.columns)
X = X[sorted_columns]
y = df.TripType

In [21]:
print(X.shape, y.shape)

(95674, 36) (95674, 2)


In [ ]:
mf = NMF()
mfX = mf.fit_transform(X.values)

In [ ]:
print(mfX.shape)
print(y.shape)

In [ ]:
XTrain, XVal, yTrain, yVal = train_test_split(mfx, y.values, test_size=0.2, random_state=200)

http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html#sklearn.grid_search.GridSearchCV

In [ ]:
start = datetime.datetime.now()
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(XTrain, yTrain)

score = clf.score(XVal, yVal)
print(score)

print("Total Time:", (datetime.datetime.now() - start).total_seconds())

In [ ]:
kh.save_validation_score(clf, score, len(XTrain))

# Only Kaggle Exists Below Here

In [ ]:
kaggleTestRaw = pd.read_csv("data/test.csv")
kaggleTest = tu.preprocess_walmart(kaggleTestRaw,
                                   group_cols, 
                                   keep_cols)
missing_columns = set(sorted_columns).difference(set(kaggleTest.columns))
print("missing columns")
print("-"*15)
for col in missing_columns:
    print(col)
    kaggleTest[col] = 0
kaggleTest = kaggleTest[sorted_columns]
assert list(X.columns) == list(kaggleTest.columns)

In [ ]:
%autoreload
test_predictions = clf.predict(kaggleTest.values)
output = tu.convert_predictions(test_predictions, 
                               output_index=pd.Series(kaggleTest.index), 
                               actual_trip_types=df.TripType.unique())

In [ ]:
kh.save_test_predictions(output, clf, len(XTrain))